In [27]:
import xenaPython as xena

host = "https://tcga.xenahubs.net"

# we will look at 5 cancers, listed here in order:
# bladder, colon, prostate, rectal, testicular
cohorts = ['BLCA','COAD','PRAD','READ','TGCT']

# we will look at 6 different datasets
# the following lists the name of each of the 6 datasets
datasets = ['gistic2','methylation450k','curated_survival_data','gene_illumina','exon_illumina', 'protein_expression_RPPA']

# compile the dataset ids for every cohort for each dataset as a dictionary
# each dictionary maps a cohort string representing a cancer type, e.g. 'TGCT' 
# with the string value representing the id for that dataset in that cohort

# dataset ids for Gistic2 dataset
dataset_gistic2 = dict([(c,'TCGA.' + c + '.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes') for c in cohorts])

# dataset ids for methlyation450k dataset
dataset_450k = dict([(c,'TCGA.' + c + '.sampleMap/HumanMethylation450') for c in cohorts])

# dataset ids for phenotype curated survival dataset
dataset_phenotype_survival = dict([(c,'survival/' + c + '_survival.txt') for c in cohorts])

# dataset ids for gene expression illumina hiseq dataset
dataset_gene_expression = dict([(c,'TCGA.' + c + '.sampleMap/HiSeqV2') for c in cohorts])

dataset_exon_expression = dict([(c,'TCGA.' + c + '.sampleMap/HiSeqV2_exon') for c in cohorts])

dataset_protein_expression = dict([(c,'TCGA.' + c + '.sampleMap/RPPA_RBN') for c in cohorts])

# collect all dictionaries of ids for different datasets and place in a list
dataset_ids_list = [dataset_gistic2,dataset_450k,dataset_phenotype_survival,
                    dataset_gene_expression,dataset_exon_expression, dataset_protein_expression]

# use the created list above to make a dictionary, where every entry is a mapping
# from a string representing a dataset, e.g. 'gistic2' with the dictionary holding
# the string ids of each gistic2 dataset in each of the 6 cancers being analysed
dataset_ids_dict = dict(zip(datasets,dataset_ids_list))

# showing what is stored in this dictionary
for dataset in dataset_ids_dict:
    print(dataset,":") # the key of the entry
    
    dataset_ids = dataset_ids_dict[dataset] # get the dictionary value mapped to this key
    
    # for each entry in this dictionary, print the key and entry
    for data_id in dataset_ids:
        print("\t",data_id,": ",dataset_ids[data_id])

gistic2 :
	 BLCA :  TCGA.BLCA.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
	 COAD :  TCGA.COAD.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
	 PRAD :  TCGA.PRAD.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
	 READ :  TCGA.READ.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
	 TGCT :  TCGA.TGCT.sampleMap/Gistic2_CopyNumber_Gistic2_all_data_by_genes
methylation450k :
	 BLCA :  TCGA.BLCA.sampleMap/HumanMethylation450
	 COAD :  TCGA.COAD.sampleMap/HumanMethylation450
	 PRAD :  TCGA.PRAD.sampleMap/HumanMethylation450
	 READ :  TCGA.READ.sampleMap/HumanMethylation450
	 TGCT :  TCGA.TGCT.sampleMap/HumanMethylation450
curated_survival_data :
	 BLCA :  survival/BLCA_survival.txt
	 COAD :  survival/COAD_survival.txt
	 PRAD :  survival/PRAD_survival.txt
	 READ :  survival/READ_survival.txt
	 TGCT :  survival/TGCT_survival.txt
gene_illumina :
	 BLCA :  TCGA.BLCA.sampleMap/HiSeqV2
	 COAD :  TCGA.COAD.sampleMap/HiSeqV2
	 PRAD :  TCGA.PRAD.sampleMap/HiSeqV2
	 READ :

In [2]:
separation_list = ['BLCA-COAD','BLCA-PRAD','BLCA-READ','BLCA-TGCT','COAD-PRAD','COAD-READ','COAD-TGCT','PRAD-READ','PRAD-TGCT','READ-TGCT']

In [3]:
from sklearn import svm

In [4]:
import numpy as np

# a function which creates the X and Y arrays needed to train a decision tree classifier with
# Parameters:
# dataset - a string representing the dataset to classify on, e.g. 'ginsic2'
# cohorts - a list of strings representing which cancer types to classify, e.g. ['TGCT','PRAD','READ']
def getXY(dataset,cohorts):
    
    X = []
    Y = []
    
    for cohort in cohorts: # for every cancer type
        
        dataset_id = dataset_ids_dict[dataset][cohort] # get the dataset id string
        samples_needed = xena.dataset_samples(host,dataset_id,200) # get a max. of 200 samples from this cohort dataset
        
        print("Number of samples is in ",cohort," is ",len(samples_needed))
         
        probes = xena.dataset_field_examples(host,dataset_id,500) 
        
        [position,vals] = xena.dataset_probe_values(host,dataset_id,samples_needed,probes) # get values
        
        print("Dimensions for vals is ",len(vals),"x",len(vals[0]))
        
        vals_array = np.array(vals)
        
        print("Dimensions of vals_array is ",len(vals_array),"x",len(vals_array[0]))
        
        vals_list = vals_array.T.tolist()
        
        print("Dimensions of vals_list is ",len(vals_list),"x",len(vals_list[0]))
        
        for j in range(len(vals_list)):
            
            X.append(vals_list[j]) # add array of length n representing n-dimensional sample
            Y.append(cohort) # add the cohort string denoting the cancer type - this is the label on the training data
            
    return X,Y

In [5]:
#  a function which returns a list of the probes associated with a given dataset and cohort, max of 500
def getProbes(dataset,cohort):
    
    dataset_id = dataset_ids_dict[dataset][cohort]
    
    probes = xena.dataset_field_examples(host,dataset_id,500)
    
    return probes

In [6]:
# a function that checks that every sample in the X array has the same length
def check_samples_equal_length(X):
    
    sample_length = len(X[0])
    
    for sample in X:
        
        if (len(sample) != sample_length): # if a sample has a different length from the first sample
            return False 
    
    print("All samples have length ",sample_length) # prints message to confirm that all samples have same length
    return True

In [7]:
#  this function replaces NaN in the X array with a 0
def replaceNAN(X):
    
    newX = []
    
    for sample in X:
        
        newSample = []
        
        for value in sample:
            if (value == "NaN"):
                newSample.append("0")
            else:
                newSample.append(value)
        
        newX.append(newSample)
    
    return newX

In [8]:
def get_max_coefficients(coefficient):
    
    max_coefficients = [0,0,0,0,0]
    indexes = [(-1,-1),(-1,-1),(-1,-1),(-1,-1),(-1,-1)]
    
    for i in range(len(coefficient)):
        
        values = coefficient[i]
        
        for j in range(len(values)):
            
            value = values[j]
            
            if (abs(value) > min(map(abs,max_coefficients))):
                
                index = list(map(abs,max_coefficients)).index(min(list(map(abs,max_coefficients))))
                max_coefficients[index] = value
                indexes[index] = (i,j)
    
    return max_coefficients,indexes
        
        
    

In [9]:
# get the X and Y values for decision tree classifier for the gistic2 dataset of all 5 cancers we are testing
X,Y = getXY('gistic2',cohorts)

Number of samples is in  BLCA  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  COAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  PRAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  READ  is  165
Dimensions for vals is  500 x 165
Dimensions of vals_array is  500 x 165
Dimensions of vals_list is  165 x 500
Number of samples is in  TGCT  is  150
Dimensions for vals is  500 x 150
Dimensions of vals_array is  500 x 150
Dimensions of vals_list is  150 x 500


In [10]:
# Checking that all samples in X are the same length
print(check_samples_equal_length(X))
print("There are ",len(X), " samples altogether")

All samples have length  500
True
There are  915  samples altogether


In [11]:
clf = svm.SVC(kernel='linear',decision_function_shape='ovo')
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
coefficients = clf.coef_
max_coefficients,associated_indexes = get_max_coefficients(coefficients)
print(max_coefficients)
print(associated_indexes)

[1.39466958523988, -1.4688413071231405, 1.5746811206610882, 1.8298656546843262, 1.5783160946878099]
[(5, 21), (5, 483), (5, 179), (5, 219), (4, 6)]


In [16]:
print(separation_list[4])
print("-------")

# get the probe values for gistic2 for each cancer

TGCT_gistic2_probes = getProbes('gistic2','TGCT')
PRAD_gistic2_probes = getProbes('gistic2','PRAD')
READ_gistic2_probes = getProbes('gistic2','READ')
COAD_gistic2_probes = getProbes('gistic2','COAD')
BLCA_gistic2_probes = getProbes('gistic2','BLCA')

# finding out what the 219th index probe value is and ensuring they're the same
print(TGCT_gistic2_probes [219])
print(PRAD_gistic2_probes [219])
print(READ_gistic2_probes [219])
print(COAD_gistic2_probes [219])
print(BLCA_gistic2_probes [219])

COAD-PRAD
-------
ACSS2
ACSS2
ACSS2
ACSS2
ACSS2


In [9]:
# getting X and Y for the gene expression data
X,Y = getXY('gene_illumina',cohorts)

Number of samples is in  BLCA  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  COAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  PRAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  READ  is  105
Dimensions for vals is  500 x 105
Dimensions of vals_array is  500 x 105
Dimensions of vals_list is  105 x 500
Number of samples is in  TGCT  is  156
Dimensions for vals is  500 x 156
Dimensions of vals_array is  500 x 156
Dimensions of vals_list is  156 x 500


In [10]:
print(check_samples_equal_length(X))
print("There are ",len(X), " samples altogether")

All samples have length  500
True
There are  861  samples altogether


In [11]:
clf = svm.SVC(kernel='linear',decision_function_shape='ovo')
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
coefficients = clf.coef_
max_coefficients,associated_indexes = get_max_coefficients(coefficients)
print(max_coefficients)
print(associated_indexes)

[0.2409001415347472, 0.1975856371559539, -0.20361075189084366, 0.21919904080966557, 0.24152112913769752]
[(5, 446), (5, 94), (5, 323), (5, 411), (5, 75)]


In [19]:
print(separation_list[5])
print("-------")

# get the probe values for gene illumina for each cancer

TGCT_gene_illumina_probes = getProbes('gene_illumina','TGCT')
PRAD_gene_illumina_probes = getProbes('gene_illumina','PRAD')
READ_gene_illumina_probes = getProbes('gene_illumina','READ')
COAD_gene_illumina_probes = getProbes('gene_illumina','COAD')
BLCA_gene_illumina_probes = getProbes('gene_illumina','BLCA')

# finding out what the 11th index probe value is and ensuring they're the same
print(TGCT_gene_illumina_probes [75])
print(PRAD_gene_illumina_probes [75])
print(READ_gene_illumina_probes [75])
print(COAD_gene_illumina_probes [75])
print(BLCA_gene_illumina_probes [75])

COAD-READ
-------
ABCB11
ABCB11
ABCB11
ABCB11
ABCB11


In [14]:
# testing the dataset methylation450k
X,Y = getXY('methylation450k',cohorts)
X = replaceNAN(X)

Number of samples is in  BLCA  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  COAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  PRAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  READ  is  106
Dimensions for vals is  500 x 106
Dimensions of vals_array is  500 x 106
Dimensions of vals_list is  106 x 500
Number of samples is in  TGCT  is  156
Dimensions for vals is  500 x 156
Dimensions of vals_array is  500 x 156
Dimensions of vals_list is  156 x 500


In [15]:
print(check_samples_equal_length(X))
print("There are ",len(X), " samples altogether")

All samples have length  500
True
There are  862  samples altogether


In [16]:
clf = svm.SVC(kernel='linear',decision_function_shape='ovo')
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
coefficients = clf.coef_
max_coefficients,associated_indexes = get_max_coefficients(coefficients)
print(max_coefficients)
print(associated_indexes)

[-1.8374963013608827, 1.165279663125819, -1.4942323557566368, 1.228063289610482, 1.2645397508705614]
[(5, 498), (5, 336), (5, 60), (5, 110), (5, 460)]


In [20]:
print(separation_list[5])
print("-------")

# get the probe values for methylation450k for each cancer

TGCT_methylation450k_probes = getProbes('methylation450k','TGCT')
PRAD_methylation450k_probes = getProbes('methylation450k','PRAD')
READ_methylation450k_probes = getProbes('methylation450k','READ')
COAD_methylation450k_probes = getProbes('methylation450k','COAD')
BLCA_methylation450k_probes = getProbes('methylation450k','BLCA')

# finding out what the 11th index probe value is and ensuring they're the same
print(TGCT_methylation450k_probes [460])
print(PRAD_methylation450k_probes [460])
print(READ_methylation450k_probes [460])
print(COAD_methylation450k_probes [460])
print(BLCA_methylation450k_probes [460])

COAD-READ
-------
cg00019118
cg00019118
cg00019118
cg00019118
cg00019118


In [21]:
# testing the dataset exon expression
X,Y = getXY('exon_illumina',cohorts)
X = replaceNAN(X)

Number of samples is in  BLCA  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  COAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  PRAD  is  200
Dimensions for vals is  500 x 200
Dimensions of vals_array is  500 x 200
Dimensions of vals_list is  200 x 500
Number of samples is in  READ  is  105
Dimensions for vals is  500 x 105
Dimensions of vals_array is  500 x 105
Dimensions of vals_list is  105 x 500
Number of samples is in  TGCT  is  156
Dimensions for vals is  500 x 156
Dimensions of vals_array is  500 x 156
Dimensions of vals_list is  156 x 500


In [22]:
print(check_samples_equal_length(X))
print("There are ",len(X), " samples altogether")

All samples have length  500
True
There are  861  samples altogether


In [23]:
clf = svm.SVC(kernel='linear',decision_function_shape='ovo')
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
coefficients = clf.coef_
max_coefficients,associated_indexes = get_max_coefficients(coefficients)
print(max_coefficients)
print(associated_indexes)

[-0.6821504905899047, -0.7767823311134023, -0.6682707600828302, 1.0101383077271038, 0.6659136295772417]
[(5, 426), (5, 251), (5, 346), (5, 383), (5, 295)]


In [25]:
print(separation_list[5])
print("-------")

# get the probe values for each cancer

TGCT_exon_illumina_probes = getProbes('exon_illumina','TGCT')
PRAD_exon_illumina_probes = getProbes('exon_illumina','PRAD')
READ_exon_illumina_probes = getProbes('exon_illumina','READ')
COAD_exon_illumina_probes = getProbes('exon_illumina','COAD')
BLCA_exon_illumina_probes = getProbes('exon_illumina','BLCA')

print(TGCT_exon_illumina_probes [383])
print(PRAD_exon_illumina_probes [383])
print(READ_exon_illumina_probes [383])
print(COAD_exon_illumina_probes [383])
print(BLCA_exon_illumina_probes [383])

COAD-READ
-------
chr10:102747070-102747272:-
chr10:102747070-102747272:-
chr10:102747070-102747272:-
chr10:102747070-102747272:-
chr10:102747070-102747272:-
